# NLP : Author Classification

In [1]:
import urllib
import random
import re
import nltk
import string
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\PCZ\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\PCZ\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\PCZ\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [1]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from xgboost import XGBClassifier
import pandas as pd

In [3]:
auth1_url1 = "https://www.gutenberg.org/files/786/786-0.txt"
auth1_url2 = "https://www.gutenberg.org/files/564/564-0.txt"
auth1_url3 = "https://www.gutenberg.org/files/675/675-0.txt"
auth1_url4 = "https://www.gutenberg.org/files/98/98-0.txt"
auth1_url5 = "https://www.gutenberg.org/files/650/650-0.txt"

In [4]:
auth2_url1 = "https://www.gutenberg.org/files/141/141-0.txt"
auth2_url2 = "https://www.gutenberg.org/cache/epub/21839/pg21839.txt"
auth2_url3 = "https://www.gutenberg.org/cache/epub/42671/pg42671.txt"
auth2_url4 = "https://www.gutenberg.org/cache/epub/105/pg105.txt"
auth2_url5 = "https://www.gutenberg.org/files/121/121-0.txt"

In [5]:
auth3_url1 = "https://www.gutenberg.org/files/108/108-0.txt"
auth3_url2 = "https://www.gutenberg.org/files/834/834-0.txt"
auth3_url3 = "https://www.gutenberg.org/files/903/903-0.txt"
auth3_url4 = "https://www.gutenberg.org/files/48320/48320-0.txt"
auth3_url5 = "https://www.gutenberg.org/files/54109/54109-0.txt"

In [6]:
auth4_url1 = "https://www.gutenberg.org/files/28289/28289-0.txt"
auth4_url2 = "https://www.gutenberg.org/cache/epub/17780/pg17780.txt"
auth4_url3 = "https://www.gutenberg.org/files/550/550-0.txt"
auth4_url4 = "https://www.gutenberg.org/cache/epub/47025/pg47025.txt"
auth4_url5 = "https://www.gutenberg.org/files/10762/10762.txt"

In [7]:
auth5_url1 = "https://www.gutenberg.org/cache/epub/60278/pg60278.txt"
auth5_url2 = "https://www.gutenberg.org/cache/epub/66837/pg66837.txt"
auth5_url3 = "https://www.gutenberg.org/files/60327/60327-0.txt"
auth5_url4 = "https://www.gutenberg.org/files/3474/3474-0.txt"
auth5_url5 = "https://www.gutenberg.org/cache/epub/27180/pg27180.txt"

In [29]:
def extract_fragments(url, L):
    #Read the file
    file = urllib.request.urlopen(url)
    lines = []
    for line in file :
        decoded_line = line.decode("utf-8")
        lines.append(decoded_line.strip())
    
    #Change the letters to lower case
    raw_docs = [doc.lower() for doc in lines]
    while ("" in raw_docs) :
        raw_docs.remove("")
    
    #Word tokenization
    tokenized_doc = [word_tokenize(doc) for doc in raw_docs]
    
    #Remove the punctuations
    regex = re.compile('[%s]'% re.escape(string.punctuation))
    tokenized_docs_no_punc = []
    for review in tokenized_doc :
        new_review = []
        for token in review :
            new_token = regex.sub(u'', token)
            if not new_token == u'' :
                new_review.append(new_token)
        tokenized_docs_no_punc.append(new_review)
    
    puncs = ["’", "‘", "“", "”"]
    for punc in puncs :
        for line in tokenized_docs_no_punc :
            while(punc in line) :
                line.remove(punc)
    
    #Remove the stop words
    tokenized_doc_no_sw = []
    for doc in tokenized_docs_no_punc :
        new_term_vector = []
        for word in doc :
            if not word in stopwords.words('english') :
                new_term_vector.append(word)
        tokenized_doc_no_sw.append(new_term_vector)
    
    lists = []
    for line in tokenized_doc_no_sw:
        lines = []
        for word in line:
            lines.append("".join(u for u in word if u not in ("—")))
        lists.append(lines)
    
    #Use the lemmantization to get the root words
    wordnet = WordNetLemmatizer()
    pre_docs = []
    for doc in lists :
        final_doc = []
        for word in doc :
            final_doc.append(wordnet.lemmatize(word))
        pre_docs.append(final_doc)
    
    #Extracting L number of fragments in size of 200
    final_list = []
    i = 199
    while len(final_list) < 200 :
            line = pre_docs[i+1]
            while len(line) <= L:
                i = i + 2
                for word in pre_docs[i]:
                    line.append(word)
            final_list.append(' '.join(line[0:L]))
    return final_list

In [30]:
L = 50

In [31]:
auth1_list1 = extract_fragments(auth1_url1, L)
auth1_list2 = extract_fragments(auth1_url2, L)
auth1_list3 = extract_fragments(auth1_url3, L)
auth1_list4 = extract_fragments(auth1_url4, L)
auth1_list5 = extract_fragments(auth1_url5, L)

In [32]:
list1 = [*auth1_list1, *auth1_list2, *auth1_list3, *auth1_list4, *auth1_list5]

In [33]:
auth2_list1 = extract_fragments(auth2_url1, L)
auth2_list2 = extract_fragments(auth2_url2, L)
auth2_list3 = extract_fragments(auth2_url3, L)
auth2_list4 = extract_fragments(auth2_url4, L)
auth2_list5 = extract_fragments(auth2_url5, L)

In [34]:
list2 = [*auth2_list1, *auth2_list2, *auth2_list3, *auth2_list4, *auth2_list5]

In [35]:
auth3_list1 = extract_fragments(auth3_url1, L)
auth3_list2 = extract_fragments(auth3_url2, L)
auth3_list3 = extract_fragments(auth3_url3, L)
auth3_list4 = extract_fragments(auth3_url4, L)
auth3_list5 = extract_fragments(auth3_url5, L)

In [36]:
list3 = [*auth3_list1, *auth3_list2, *auth3_list3, *auth3_list4, *auth3_list5]

In [37]:
auth4_list1 = extract_fragments(auth4_url1, L)
auth4_list2 = extract_fragments(auth4_url2, L)
auth4_list3 = extract_fragments(auth4_url3, L)
auth4_list4 = extract_fragments(auth4_url4, L)
auth4_list5 = extract_fragments(auth4_url5, L)

In [38]:
list4 = [*auth4_list1, *auth4_list2, *auth4_list3, *auth4_list4, *auth4_list5]

In [39]:
auth5_list1 = extract_fragments(auth5_url1, L)
auth5_list2 = extract_fragments(auth5_url2, L)
auth5_list3 = extract_fragments(auth5_url3, L)
auth5_list4 = extract_fragments(auth5_url4, L)
auth5_list5 = extract_fragments(auth5_url5, L)

In [40]:
list5 = [*auth5_list1, *auth5_list2, *auth5_list3, *auth5_list4, *auth5_list5]

In [71]:
x = []
x = [*list1, *list2, *list3, *list4, *list5]

In [72]:
len(x), x[0]

(5000,
 'beginning sunbeam bitzer corner row side row advance caught end receive deeper lustrous colour sun shone ray appeared draw little colour ever possessed lash bringing immediate contrast something might mere continuation sandy freckle forehead looked though cut would bleed white quadruped graminivorous forty teeth namely twentyfour grinder country shed hoof')

In [167]:
authors = ['Charles Dickens', 'Jane Austen', 'Sir Arthur Conan Doyle', 'George Eliot', 'Hugh Walpole']
target = []
for author in authors :
    for i in range(0, 1000):
        target.append(author)
y = LabelEncoder().fit_transform(target)

In [168]:
len(y), set(y)

(5000, {0, 1, 2, 3, 4})

In [169]:
data_df = pd.DataFrame({
    'Extracted Fragments' : [],
    'Author' : []
})

In [170]:
for i in range (0, len(x)) :
    data_df = data_df.append({
        'Extracted Fragments' : x[i],
        'Author' : target[i],
    }, ignore_index = True
    )

In [171]:
data_df

,Extracted Fragments,Author
0,beginning sunbeam bitzer corner row side row a...,Charles Dickens
1,iron age known mark mouth thus much bitzer gir...,Charles Dickens
2,continue fistic phraseology genius coming scra...,Charles Dickens
3,ventured answer paper room would paint must pa...,Charles Dickens
4,general conviction time sir always feeble stra...,Charles Dickens
...,...,...
4995,robin rate made meaning clear wish regard comp...,Hugh Walpole
4996,omitted mine came back stranger ready anything...,Hugh Walpole
4997,see please shall course disguise position worl...,Hugh Walpole
4998,end would father said show stiff back white mo...,Hugh Walpole


In [172]:
data_df.to_csv('Dataset_L50.csv')

## BOW

### Logistic Regression

In [91]:
bow = CountVectorizer()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train_bow = bow.fit_transform(x_train)

In [84]:
lr_par = {
    'penalty' : ['l2'],
    'C' : [0.1, 1.0, 10.0],
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'max_iter' : [100, 200, 300]
}

In [85]:
LR = LogisticRegression()
lr_grid = GridSearchCV(LR, lr_par, refit = True, n_jobs=-1, cv=5)
lr_grid.fit(x_train_bow, y_train)
opt_par = lr_grid.best_params_
opt_par

{'C': 10.0, 'max_iter': 100, 'penalty': 'l2', 'solver': 'newton-cg'}

In [87]:
lr_train_acc_bow = []
lr_test_acc_bow = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    LR = LogisticRegression(C = 10.0, max_iter = 100, penalty = 'l2', solver = 'newton-cg')
    LR.fit(x_train_bow, y_train)
    y_pred = LR.predict(x_test_bow)
    y_train_pred = LR.predict(x_train_bow)
    lr_train_acc_bow.append(metrics.accuracy_score(y_train, y_train_pred))
    lr_test_acc_bow.append(metrics.accuracy_score(y_test, y_pred))

In [88]:
lr_mean_train_acc_bow = sum(lr_train_acc_bow)/len(lr_train_acc_bow)
lr_mean_test_acc_bow = sum(lr_test_acc_bow)/len(lr_test_acc_bow)

In [89]:
lr_mean_train_acc_bow, lr_mean_test_acc_bow

(1.0, 0.9194000000000001)

### Support Vector Machines

In [94]:
bow = CountVectorizer()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_bow = bow.fit_transform(x_train)

In [92]:
svm_par = [
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['linear']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['poly'],
    'degree' : [2, 3, 4],
    'gamma' : ['scale', 'auto']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['rbf', 'sigmoid'],
    'gamma' : ['scale', 'auto']
    }
]

In [93]:
SVM = SVC()
svm_grid = GridSearchCV(SVM, svm_par, refit = True, n_jobs=-1, cv=5)
svm_grid.fit(x_train_bow, y_train)
opt_par = svm_grid.best_params_
opt_par

{'C': 0.1, 'kernel': 'linear'}

In [96]:
svm_train_acc_bow = []
svm_test_acc_bow = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    SVM = SVC( C = 0.1, kernel = 'linear' )
    SVM.fit(x_train_bow, y_train)
    y_pred = SVM.predict(x_test_bow)
    y_train_pred = SVM.predict(x_train_bow)
    svm_train_acc_bow.append(metrics.accuracy_score(y_train, y_train_pred))
    svm_test_acc_bow.append(metrics.accuracy_score(y_test, y_pred))

In [97]:
svm_mean_train_acc_bow = sum(svm_train_acc_bow)/len(svm_train_acc_bow)
svm_mean_test_acc_bow = sum(svm_test_acc_bow)/len(svm_test_acc_bow)

In [101]:
svm_mean_train_acc_bow, svm_mean_test_acc_bow

(1.0, 0.9016499999999998)

### Random Forest

In [99]:
bow = CountVectorizer()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_bow = bow.fit_transform(x_train)

In [100]:
rf_par = {
    'n_estimators' : [50, 100, 200],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [8, 16, 32],
    'max_features' : ['sqrt', 'log2']
}

In [101]:
RF = RandomForestClassifier()
rf_grid = GridSearchCV(RF, rf_par, refit = True, n_jobs=-1, cv=5)
rf_grid.fit(x_train_bow, y_train)
opt_par = rf_grid.best_params_
opt_par

{'criterion': 'gini',
 'max_depth': 32,
 'max_features': 'log2',
 'n_estimators': 200}

In [103]:
rf_train_acc_bow = []
rf_test_acc_bow = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    RF = RandomForestClassifier(criterion = 'gini', max_depth = 32, max_features = 'log2', n_estimators = 200 )
    RF.fit(x_train_bow, y_train)
    y_pred = RF.predict(x_test_bow)
    y_train_pred = RF.predict(x_train_bow)
    rf_train_acc_bow.append(metrics.accuracy_score(y_train, y_train_pred))
    rf_test_acc_bow.append(metrics.accuracy_score(y_test, y_pred))

In [104]:
rf_mean_train_acc_bow = sum(rf_train_acc_bow)/len(rf_train_acc_bow)
rf_mean_test_acc_bow = sum(rf_test_acc_bow)/len(rf_test_acc_bow)

In [105]:
rf_mean_train_acc_bow, rf_mean_test_acc_bow

(0.9998250000000001, 0.86775)

### Naive Bays 

In [109]:
nb_train_acc_bow = []
nb_test_acc_bow = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    NB = MultinomialNB()
    NB.fit(x_train_bow, y_train)
    y_pred = NB.predict(x_test_bow)
    y_train_pred = NB.predict(x_train_bow)
    nb_train_acc_bow.append(metrics.accuracy_score(y_train, y_train_pred))
    nb_test_acc_bow.append(metrics.accuracy_score(y_test, y_pred))

In [110]:
nb_mean_train_acc_bow = sum(nb_train_acc_bow)/len(nb_train_acc_bow)
nb_mean_test_acc_bow = sum(nb_test_acc_bow)/len(nb_test_acc_bow)

In [111]:
nb_mean_train_acc_bow, nb_mean_test_acc_bow

(0.991, 0.9328000000000001)

### XGBoost

In [112]:
bow = CountVectorizer()
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_bow = bow.fit_transform(x_train)

In [113]:
xgb_par = {
    'n_estimators' : [20, 50, 100],
    'use_label_encoder' : [False],
    'max_depth' : [8, 16, 32],
    'learning_rate' : [0.01, 0.1]
}

In [114]:
XGB = XGBClassifier()
xgb_grid = GridSearchCV(XGB, xgb_par, refit = True, n_jobs=-1, cv=5)
xgb_grid.fit(x_train_bow, y_train)
opt_par = xgb_grid.best_params_
opt_par

[20:58:49] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


{'learning_rate': 0.1,
 'max_depth': 8,
 'n_estimators': 100,
 'use_label_encoder': False}

In [116]:
xgb_train_acc_bow = []
xgb_test_acc_bow = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    bow = CountVectorizer()
    x_train_bow = bow.fit_transform(x_train)
    x_test_bow = bow.transform(x_test)
    XGB = XGBClassifier(learning_rate = 0.1, max_depth = 8, n_estimators = 100, use_label_encoder = False )
    XGB.fit(x_train_bow, y_train)
    y_pred = XGB.predict(x_test_bow)
    y_train_pred = XGB.predict(x_train_bow)
    xgb_train_acc_bow.append(metrics.accuracy_score(y_train, y_train_pred))
    xgb_test_acc_bow.append(metrics.accuracy_score(y_test, y_pred))

[15:25:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:26:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:26:24] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:26:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [117]:
xgb_mean_train_acc_bow = sum(xgb_train_acc_bow)/len(xgb_train_acc_bow)
xgb_mean_test_acc_bow = sum(xgb_test_acc_bow)/len(xgb_test_acc_bow)

In [118]:
xgb_mean_train_acc_bow, xgb_mean_test_acc_bow

(0.984675, 0.8283499999999998)

In [174]:
algorithms = ['Logistic Regression', 'Support Vector Machine', 'Random Forest', 'Naive Bayes', 'XGBoost']

In [175]:
bow_df = pd.DataFrame({
    'Algorithm' : [], 
    'Train Accuracies' : [],
    'Mean Train Accuracy' : [],
    'Test Accuracies' : [],
    'Mean Test Accuracy' :[]
})

In [176]:
train_acc_bow = [lr_train_acc_bow, svm_train_acc_bow, rf_train_acc_bow,
                      nb_train_acc_bow, xgb_train_acc_bow]
test_acc_bow = [lr_test_acc_bow, svm_test_acc_bow, rf_test_acc_bow,
                    nb_test_acc_bow, xgb_test_acc_bow]

In [177]:
mean_train_acc_bow = [lr_mean_train_acc_bow, svm_mean_train_acc_bow, rf_mean_train_acc_bow,
                      nb_mean_train_acc_bow, xgb_mean_train_acc_bow]
mean_test_acc_bow = [lr_mean_test_acc_bow, svm_mean_test_acc_bow, rf_mean_test_acc_bow,
                    nb_mean_test_acc_bow, xgb_mean_test_acc_bow]

In [178]:
for i in range (0, len(algorithms)) :
    bow_df = bow_df.append({
        'Algorithm' : algorithms[i],
        'Train Accuracies' : train_acc_bow[i],
        'Mean Train Accuracy' : mean_train_acc_bow[i],
        'Test Accuracies' : test_acc_bow[i],
        'Mean Test Accuracy' : mean_test_acc_bow[i],
    }, ignore_index = True
    )

In [179]:
bow_df

,Algorithm,Train Accuracies,Mean Train Accuracy,Test Accuracies,Mean Test Accuracy
0,Logistic Regression,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,"[0.922, 0.915, 0.902, 0.925, 0.907, 0.922, 0.9...",0.91940
1,Support Vector Machine,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,"[0.906, 0.901, 0.879, 0.912, 0.887, 0.897, 0.9...",0.90165
2,Random Forest,"[0.99975, 1.0, 1.0, 1.0, 0.99975, 0.99925, 1.0...",0.999825,"[0.87, 0.878, 0.861, 0.876, 0.869, 0.852, 0.86...",0.86775
3,Naive Bayes,"[0.992, 0.99175, 0.992, 0.99075, 0.9905, 0.991...",0.991000,"[0.939, 0.946, 0.928, 0.937, 0.934, 0.93, 0.92...",0.93280
4,XGBoost,"[0.984, 0.985, 0.9865, 0.98525, 0.98375, 0.985...",0.984675,"[0.841, 0.828, 0.817, 0.832, 0.826, 0.82, 0.82...",0.82835


In [180]:
bow_df.to_csv('BOW_Results.csv')

## Tf-idf

### Logistic Regression

In [125]:
tf_idf = TfidfVectorizer(use_idf=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_tf = tf_idf.fit_transform(x_train)

In [126]:
lr_par = {
    'penalty' : ['l2'],
    'C' : [0.1, 1.0, 10.0],
    'solver' : ['newton-cg', 'lbfgs', 'sag', 'saga'],
    'max_iter' : [100, 200, 300]
}

In [127]:
LR = LogisticRegression()
lr_grid = GridSearchCV(LR, lr_par, refit = True, n_jobs=-1, cv=5)
lr_grid.fit(x_train_tf, y_train)
opt_par = lr_grid.best_params_
opt_par

{'C': 10.0, 'max_iter': 200, 'penalty': 'l2', 'solver': 'saga'}

In [129]:
lr_train_acc_tf = []
lr_test_acc_tf = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    LR = LogisticRegression(C = 10.0, max_iter = 200, penalty = 'l2', solver = 'saga')
    LR.fit(x_train_tf, y_train)
    y_pred = LR.predict(x_test_tf)
    y_train_pred = LR.predict(x_train_tf)
    lr_train_acc_tf.append(metrics.accuracy_score(y_train, y_train_pred))
    lr_test_acc_tf.append(metrics.accuracy_score(y_test, y_pred))

In [130]:
lr_mean_train_acc_tf = sum(lr_train_acc_tf)/len(lr_train_acc_tf)
lr_mean_test_acc_tf = sum(lr_test_acc_tf)/len(lr_test_acc_tf)

In [131]:
lr_mean_train_acc_tf, lr_mean_test_acc_tf

(1.0, 0.9442499999999999)

### Support Vector Machine

In [132]:
tf_idf = TfidfVectorizer(use_idf=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_tf = tf_idf.fit_transform(x_train)

In [133]:
svm_par = [
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['linear']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['poly'],
    'degree' : [2, 3, 4],
    'gamma' : ['scale', 'auto']
   },
   { 'C' : [0.1, 1.0, 10.0],
    'kernel' : ['rbf', 'sigmoid'],
    'gamma' : ['scale', 'auto']
    }
]

In [134]:
SVM = SVC()
svm_grid = GridSearchCV(SVM, svm_par, refit = True, n_jobs=-1, cv=5)
svm_grid.fit(x_train_tf, y_train)
opt_par = svm_grid.best_params_
opt_par

{'C': 1.0, 'kernel': 'linear'}

In [136]:
svm_train_acc_tf = []
svm_test_acc_tf = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    SVM = SVC( C = 1.0, kernel = 'linear' )
    SVM.fit(x_train_tf, y_train)
    y_pred = SVM.predict(x_test_tf)
    y_train_pred = SVM.predict(x_train_tf)
    svm_train_acc_tf.append(metrics.accuracy_score(y_train, y_train_pred))
    svm_test_acc_tf.append(metrics.accuracy_score(y_test, y_pred))

In [137]:
svm_mean_train_acc_tf = sum(svm_train_acc_tf)/len(svm_train_acc_tf)
svm_mean_test_acc_tf = sum(svm_test_acc_tf)/len(svm_test_acc_tf)

In [138]:
svm_mean_train_acc_tf, svm_mean_test_acc_tf

(0.9992124999999998, 0.9399999999999998)

### Random Forest

In [139]:
tf_idf = TfidfVectorizer(use_idf=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
x_train_tf = tf_idf.fit_transform(x_train)

In [140]:
rf_par = {
    'n_estimators' : [50, 100, 200],
    'criterion' : ['gini', 'entropy'],
    'max_depth' : [8, 16, 32],
    'max_features' : ['sqrt', 'log2']
}

In [141]:
RF = RandomForestClassifier()
rf_grid = GridSearchCV(RF, rf_par, refit = True, n_jobs=-1, cv=5)
rf_grid.fit(x_train_tf, y_train)
opt_par = rf_grid.best_params_
opt_par

{'criterion': 'gini',
 'max_depth': 32,
 'max_features': 'log2',
 'n_estimators': 200}

In [143]:
rf_train_acc_tf = []
rf_test_acc_tf = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    RF = RandomForestClassifier(criterion = 'gini', max_depth = 32, max_features = 'log2', n_estimators = 200)
    RF.fit(x_train_tf, y_train)
    y_pred = RF.predict(x_test_tf)
    y_train_pred = RF.predict(x_train_tf)
    rf_train_acc_tf.append(metrics.accuracy_score(y_train, y_train_pred))
    rf_test_acc_tf.append(metrics.accuracy_score(y_test, y_pred))

In [144]:
rf_mean_train_acc_tf = sum(rf_train_acc_tf)/len(rf_train_acc_tf)
rf_mean_test_acc_tf = sum(rf_test_acc_tf)/len(rf_test_acc_tf)

In [145]:
rf_mean_train_acc_tf, rf_mean_test_acc_tf

(0.9893375000000001, 0.788)

### Naive Bayes

In [151]:
nb_train_acc_tf = []
nb_test_acc_tf = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    NB = MultinomialNB()
    NB.fit(x_train_tf, y_train)
    y_pred = NB.predict(x_test_tf)
    y_train_pred = NB.predict(x_train_tf)
    nb_train_acc_tf.append(metrics.accuracy_score(y_train, y_train_pred))
    nb_test_acc_tf.append(metrics.accuracy_score(y_test, y_pred))

In [152]:
nb_mean_train_acc_tf = sum(nb_train_acc_tf)/len(nb_train_acc_tf)
nb_mean_test_acc_tf = sum(nb_test_acc_tf)/len(nb_test_acc_tf)

In [153]:
nb_mean_train_acc_tf, nb_mean_test_acc_tf

(0.9757000000000001, 0.8935000000000001)

### XGBoost

In [154]:
tf_idf = TfidfVectorizer(use_idf=True)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
x_train_tf = tf_idf.fit_transform(x_train)

In [155]:
xgb_par = {
    'n_estimators' : [20, 50, 100],
    'use_label_encoder' : [False],
    'max_depth' : [8, 16, 32],
    'learning_rate' : [0.01, 0.1]
}

In [156]:
XGB = XGBClassifier()
xgb_grid = GridSearchCV(XGB, xgb_par, refit = True, n_jobs=-1, cv=5)
xgb_grid.fit(x_train_tf, y_train)
opt_par = xgb_grid.best_params_
opt_par

[22:43:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


{'learning_rate': 0.1,
 'max_depth': 8,
 'n_estimators': 100,
 'use_label_encoder': False}

In [158]:
xgb_train_acc_tf = []
xgb_test_acc_tf = []
for i in range (0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1)
    tf_idf = TfidfVectorizer(use_idf=True)
    x_train_tf = tf_idf.fit_transform(x_train)
    x_test_tf = tf_idf.transform(x_test)
    XGB = XGBClassifier(learning_rate = 0.1, max_depth = 8, n_estimators = 100, use_label_encoder = False)
    XGB.fit(x_train_tf, y_train)
    y_pred = XGB.predict(x_test_tf)
    y_train_pred = XGB.predict(x_train_tf)
    xgb_train_acc_tf.append(metrics.accuracy_score(y_train, y_train_pred))
    xgb_test_acc_tf.append(metrics.accuracy_score(y_test, y_pred))

[15:49:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:50:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:51:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.0/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[15:52:10] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [159]:
xgb_mean_train_acc_tf = sum(xgb_train_acc_tf)/len(xgb_train_acc_tf)
xgb_mean_test_acc_tf = sum(xgb_test_acc_tf)/len(xgb_test_acc_tf)

In [160]:
xgb_mean_train_acc_tf, xgb_mean_test_acc_tf

(0.9883624999999998, 0.8030000000000002)

In [181]:
algorithms = ['Logistic Regression', 'Support Vector Machine', 'Random Forest', 'Naive Bayes', 'XGBoost']

In [182]:
tf_df = pd.DataFrame({
    'Algorithm' : [],
    'Train Accuracies' : [],
    'Mean Train Accuracy' : [], 
    'Test Accuracies' : [],
    'Mean Test Accuracy' :[]
})

In [183]:
train_acc_tf = [lr_train_acc_tf, svm_train_acc_tf, rf_train_acc_tf,
                      nb_train_acc_tf, xgb_train_acc_tf]
test_acc_tf = [lr_test_acc_tf, svm_test_acc_tf, rf_test_acc_tf,
                    nb_test_acc_tf, xgb_test_acc_tf]

In [184]:
mean_train_acc_tf = [lr_mean_train_acc_tf, svm_mean_train_acc_tf, rf_mean_train_acc_tf,
                      nb_mean_train_acc_tf, xgb_mean_train_acc_tf]
mean_test_acc_tf = [lr_mean_test_acc_tf, svm_mean_test_acc_tf, rf_mean_test_acc_tf,
                    nb_mean_test_acc_tf, xgb_mean_test_acc_tf]

In [185]:
for i in range (0, len(algorithms)) :
    tf_df = tf_df.append({
        'Algorithm' : algorithms[i],
        'Train Accuracies' : train_acc_tf[i],
        'Mean Train Accuracy' : mean_train_acc_tf[i],
        'Test Accuracies' : test_acc_tf[i],
        'Mean Test Accuracy' : mean_test_acc_tf[i],
    }, ignore_index = True
    )

In [186]:
tf_df

,Algorithm,Train Accuracies,Mean Train Accuracy,Test Accuracies,Mean Test Accuracy
0,Logistic Regression,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...",1.000000,"[0.949, 0.943, 0.93, 0.944, 0.935, 0.949, 0.94...",0.94425
1,Support Vector Machine,"[0.99875, 0.99975, 0.99875, 0.99875, 0.9995, 0...",0.999212,"[0.943, 0.946, 0.931, 0.947, 0.935, 0.935, 0.9...",0.94000
2,Random Forest,"[0.989, 0.988, 0.98825, 0.9895, 0.9905, 0.9897...",0.989338,"[0.813, 0.781, 0.779, 0.809, 0.787, 0.783, 0.7...",0.78800
3,Naive Bayes,"[0.978, 0.977, 0.974, 0.97575, 0.974, 0.96975,...",0.975700,"[0.906, 0.896, 0.886, 0.902, 0.891, 0.886, 0.9...",0.89350
4,XGBoost,"[0.9865, 0.99, 0.988, 0.9885, 0.98925, 0.98925...",0.988362,"[0.809, 0.804, 0.794, 0.808, 0.795, 0.8, 0.806...",0.80300


In [187]:
tf_df.to_csv('TF-IDF_Results.csv')

## Loading Data

In [2]:
data_df = pd.read_csv('Dataset_L50.csv')

In [3]:
x = data_df['Extracted Fragments'].values
y = LabelEncoder().fit_transform(data_df['Author'])
y_enc = pd.get_dummies(data_df['Author'])

In [4]:
len(x), len(y), len(y_enc)

(5000, 5000, 5000)

In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, LSTM, Bidirectional
from keras.layers.embeddings import Embedding
from keras.callbacks import EarlyStopping
from keras.initializers import Constant
from gensim.models import KeyedVectors
from sklearn.metrics import accuracy_score
import numpy as np

## Word2Vec

In [2]:
word2vec_model = KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary = True )

In [3]:
embedding_dim = 300

### LSTM

In [13]:
lstm_train_acc_wv = []
lstm_val_acc_wv = []
lstm_test_acc_wv = []
for i in range (0, 20) :
    x_train, x_test, y_train, y_test = train_test_split(x, y_enc, test_size = 0.15, random_state = i + 1)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x_train)
    x_train_seq  = tokenizer.texts_to_sequences(x_train) 
    x_test_seq = tokenizer.texts_to_sequences(x_test)
    max_len = max([len(x) for x in x_train_seq])
    x_train_pad  = pad_sequences(x_train_seq, padding = 'post', maxlen = max_len)
    x_test_pad = pad_sequences(x_test_seq, padding = 'post', maxlen = max_len)
    vocab_size = len(tokenizer.word_index) + 1
    word_index = tokenizer.word_index
    
    embedding_matrix = np.zeros((len(word_index) + 1, embedding_dim))
    for word, i in word_index.items():
        if word in word2vec_model: 
            embedding_vector = word2vec_model[word]
            embedding_matrix[i] = embedding_vector
    
    embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    weights = [embedding_matrix],
    input_length = max_len,
    trainable = False
    )
    
    model_wv = Sequential()
    model_wv.add(embedding_layer)
    model_wv.add(LSTM(units = 64))
    model_wv.add(Dropout(0.4))
    model_wv.add(Dense(100, activation = 'tanh'))
    model_wv.add(Dropout(0.4))
    model_wv.add(Dense(5, activation = 'softmax'))

    model_wv.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    results_wv_lstm = model_wv.fit(
        x_train_pad, 
        y_train,
        batch_size = 8,
        epochs = 10, 
        validation_split = 0.2,
        verbose = 0
    )
    
    epoch = len(results_wv_lstm.history['accuracy'])
    lstm_train_acc_wv.append(results_wv_lstm.history['accuracy'][epoch - 1])
    lstm_val_acc_wv.append(results_wv_lstm.history['val_accuracy'][epoch - 1])
    
    pred = model_wv.predict(x_test_pad)
    predicted_labels = np.argmax ( pred, axis = -1 )
    y_test_array = np.argmax ( np.array(y_test), axis = -1 )
    test_accuracy = accuracy_score ( y_test_array, predicted_labels )
    lstm_test_acc_wv.append(test_accuracy)

In [14]:
lstm_mean_train_acc_wv = sum(lstm_train_acc_wv)/len(lstm_train_acc_wv)
lstm_mean_val_acc_wv = sum(lstm_val_acc_wv)/len(lstm_val_acc_wv)
lstm_mean_test_acc_wv = sum(lstm_test_acc_wv)/len(lstm_test_acc_wv)

In [15]:
lstm_mean_train_acc_wv, lstm_mean_val_acc_wv, lstm_mean_test_acc_wv

(0.916411766409874, 0.7609999895095825, 0.7664)

## GLoVe

In [45]:
embedding_index = {}
file = open('glove.6B.300d.txt', encoding = "utf-8")
for line in file:
    values = line.split()
    word = values[0]
    coeff = np.asarray(values[1:], dtype='float32')
    embedding_index[word] = coeff
file.close()

In [46]:
embedding_dim = 300

### GRU

In [103]:
gru_train_acc_glv = []
gru_val_acc_glv = []
gru_test_acc_glv = []

for i in range (0, 20) :
    x_train, x_test, y_train, y_test = train_test_split(x, y_enc, test_size = 0.2, random_state = i + 1)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x_train)
    x_train_seq  = tokenizer.texts_to_sequences(x_train) 
    x_test_seq = tokenizer.texts_to_sequences(x_test)
    max_len = max([len(x) for x in x_train_seq])
    x_train_pad  = pad_sequences(x_train_seq, padding = 'post', maxlen = max_len)
    x_test_pad = pad_sequences(x_test_seq, padding = 'post', maxlen = max_len)
    vocab_size = len(tokenizer.word_index) + 1
    word_index = tokenizer.word_index
            
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    
    embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    weights = [embedding_matrix],
    input_length = max_len,
    trainable = False
    )
    
    model_glv = Sequential()
    model_glv.add(embedding_layer)
    model_glv.add(GRU(units = 32))
    model_glv.add(Dense(5, activation = 'softmax'))

    model_glv.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    results_glv_gru = model_glv.fit(
        x_train_pad, 
        y_train, 
        epochs = 10, 
        validation_split = 0.2,
        verbose = 0
    )
    
    epoch = len(results_glv_gru.history['accuracy'])
    gru_train_acc_glv.append(results_glv_gru.history['accuracy'][epoch-1])
    gru_val_acc_glv.append(results_glv_gru.history['val_accuracy'][epoch-1])
    
    pred = model_glv.predict(x_test_pad)
    predicted_labels = np.argmax ( pred, axis = -1 )
    y_test_array = np.argmax ( np.array(y_test), axis = -1 )
    test_accuracy = accuracy_score ( y_test_array, predicted_labels )
    gru_test_acc_glv.append(test_accuracy)

In [104]:
gru_mean_train_acc_glv = sum(gru_train_acc_glv)/len(gru_train_acc_glv)
gru_mean_val_acc_glv = sum(gru_val_acc_glv)/len(gru_val_acc_glv)
gru_mean_test_acc_glv = sum(gru_test_acc_glv)/len(gru_test_acc_glv)

In [105]:
gru_mean_train_acc_glv, gru_mean_val_acc_glv, gru_mean_test_acc_glv

(0.9694531202316284, 0.8203125, 0.8190999999999999)

### LSTM

In [100]:
lstm_train_acc_glv = []
lstm_val_acc_glv = []
lstm_test_acc_glv = []
for i in range (0, 20) :
    x_train, x_test, y_train, y_test = train_test_split(x, y_enc, test_size = 0.2, random_state = i + 1)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x_train)
    x_train_seq  = tokenizer.texts_to_sequences(x_train) 
    x_test_seq = tokenizer.texts_to_sequences(x_test)
    max_len = max([len(x) for x in x_train_seq])
    x_train_pad  = pad_sequences(x_train_seq, padding = 'post', maxlen = max_len)
    x_test_pad = pad_sequences(x_test_seq, padding = 'post', maxlen = max_len)
    vocab_size = len(tokenizer.word_index) + 1
    word_index = tokenizer.word_index
            
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    
    embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    weights = [embedding_matrix],
    input_length = max_len,
    trainable = False
    )
    
    model_glv = Sequential()
    model_glv.add(embedding_layer)
    model_glv.add(LSTM(units = 32))
    model_glv.add(Dense(5, activation = 'softmax'))

    model_glv.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    results_glv_lstm = model_glv.fit(
        x_train_pad, 
        y_train, 
        epochs = 10, 
        validation_split = 0.2,
        verbose = 0
    )
    
    epoch = len(results_glv_lstm.history['accuracy'])
    lstm_train_acc_glv.append(results_glv_lstm.history['accuracy'][epoch-1])
    lstm_val_acc_glv.append(results_glv_lstm.history['val_accuracy'][epoch-1])
    
    pred = model_glv.predict(x_test_pad)
    predicted_labels = np.argmax ( pred, axis = -1 )
    y_test_array = np.argmax ( np.array(y_test), axis = -1 )
    test_accuracy = accuracy_score ( y_test_array, predicted_labels )
    lstm_test_acc_glv.append(test_accuracy)

In [101]:
lstm_mean_train_acc_glv = sum(lstm_train_acc_glv)/len(lstm_train_acc_glv)
lstm_mean_val_acc_glv = sum(lstm_val_acc_glv)/len(lstm_val_acc_glv)
lstm_mean_test_acc_glv = sum(lstm_test_acc_glv)/len(lstm_test_acc_glv)

In [102]:
lstm_mean_train_acc_glv, lstm_mean_val_acc_glv, lstm_mean_test_acc_glv

(0.9628437578678131, 0.7983749985694886, 0.7925000000000001)

### Bidirectional LSTM

In [50]:
blstm_train_acc_glv = []
blstm_val_acc_glv = []
blstm_test_acc_glv = []
for i in range (0, 20) :
    x_train, x_test, y_train, y_test = train_test_split(x, y_enc, test_size = 0.2, random_state = i + 1)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x_train)
    x_train_seq  = tokenizer.texts_to_sequences(x_train) 
    x_test_seq = tokenizer.texts_to_sequences(x_test)
    max_len = max([len(x) for x in x_train_seq])
    x_train_pad  = pad_sequences(x_train_seq, padding = 'post', maxlen = max_len)
    x_test_pad = pad_sequences(x_test_seq, padding = 'post', maxlen = max_len)
    vocab_size = len(tokenizer.word_index) + 1
    word_index = tokenizer.word_index
            
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, i in word_index.items():
        embedding_vector = embedding_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    
    embedding_layer = Embedding(
    vocab_size,
    embedding_dim,
    weights = [embedding_matrix],
    input_length = max_len,
    trainable = False
    )
    
    model_glv = Sequential()
    model_glv.add(embedding_layer)
    model_glv.add(Bidirectional(LSTM(units = 32)))
    model_glv.add(Dropout(0.4))
    model_glv.add(Dense(5, activation = 'softmax'))

    model_glv.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
    results_glv_blstm = model_glv.fit(
        x_train_pad, 
        y_train, 
        epochs = 10, 
        validation_split = 0.2,
        verbose = 0
    )
    
    epoch = len(results_glv_blstm.history['accuracy'])
    blstm_train_acc_glv.append(results_glv_blstm.history['accuracy'][epoch-1])
    blstm_val_acc_glv.append(results_glv_blstm.history['val_accuracy'][epoch-1])
    
    pred = model_glv.predict(x_test_pad)
    predicted_labels = np.argmax ( pred, axis = -1 )
    y_test_array = np.argmax ( np.array(y_test), axis = -1 )
    test_accuracy = accuracy_score ( y_test_array, predicted_labels )
    blstm_test_acc_glv.append(test_accuracy)

In [51]:
blstm_mean_train_acc_glv = sum(blstm_train_acc_glv)/len(blstm_train_acc_glv)
blstm_mean_val_acc_glv = sum(blstm_val_acc_glv)/len(blstm_val_acc_glv)
blstm_mean_test_acc_glv = sum(blstm_test_acc_glv)/len(blstm_test_acc_glv)

In [52]:
blstm_mean_train_acc_glv, blstm_mean_val_acc_glv, blstm_mean_test_acc_glv

(0.9682031273841858, 0.8087499976158142, 0.8086499999999999)

In [55]:
models = ['GRU', 'LSTM', 'Bidirectional LSTM']
train_acc_glv = [gru_mean_train_acc_glv, lstm_mean_train_acc_glv, blstm_mean_train_acc_glv]
val_acc_glv = [gru_mean_val_acc_glv, lstm_mean_val_acc_glv, blstm_mean_val_acc_glv]
test_acc_glv = [gru_mean_test_acc_glv, lstm_mean_test_acc_glv, blstm_mean_test_acc_glv]

In [56]:
glove_df = pd.DataFrame({
    'Model' : models,
    'Mean Train Accuracy' : train_acc_glv,
    'Mean Validation Accuracy' : val_acc_glv,
    'Mean Test Accuracy' : test_acc_glv
})

In [57]:
glove_df

,Model,Mean Train Accuracy,Mean Validation Accuracy,Mean Test Accuracy
0,GRU,0.969453,0.820312,0.81910
1,LSTM,0.962844,0.798375,0.79250
2,Bidirectional LSTM,0.968203,0.808750,0.80865


In [58]:
glove_df.to_csv('Glove_Results.csv')

## Transformer

In [5]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [6]:
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate = 0.1):
        super(TransformerBlock, self).__init__()
        self.att = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation = "relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

In [7]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start = 0, limit = maxlen, delta = 1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

In [11]:
embed_dim = 32 
num_heads = 2
ff_dim = 32  

In [114]:
tf_train_acc, tf_val_acc,tf_test_acc = [], [], []
for i in range(0, 20):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = i+1 )
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(x_train)
    x_train_seq  = tokenizer.texts_to_sequences(x_train) 
    x_test_seq = tokenizer.texts_to_sequences(x_test)
    max_len = max([len(x) for x in x_train_seq])
    x_train_pad  = pad_sequences(x_train_seq, padding = 'post', maxlen = max_len)
    x_test_pad = pad_sequences(x_test_seq, padding = 'post', maxlen = max_len)
    vocab_size = len(tokenizer.word_index) + 1
    
    inputs = layers.Input(shape=(max_len,))
    embedding_layer = TokenAndPositionEmbedding(max_len, vocab_size, embed_dim)
    x_ = embedding_layer(inputs)
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x_ = transformer_block(x_)
    x_ = layers.GlobalAveragePooling1D()(x_)
    x_ = layers.Dropout(0.2)(x_)
    x_ = layers.Dense(50, activation="relu")(x_)
    x_ = layers.Dropout(0.2)(x_)
    outputs = layers.Dense(5, activation="softmax")(x_)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    
    model.compile(optimizer = "adam", loss = "sparse_categorical_crossentropy", metrics = ["accuracy"])

    results_tf = model.fit(
        x_train_pad, y_train, batch_size = 16, epochs = 3, validation_split = 0.15, verbose = 0
    )
    tf_train_acc.append(results_tf.history['accuracy'][2])
    tf_val_acc.append(results_tf.history['val_accuracy'][2])
    
    pred = model.predict(x_test_pad)
    predicted_labels = np.argmax ( pred, axis = -1 )
    tf_test_acc.append(accuracy_score ( y_test, predicted_labels ))

In [13]:
tf_mean_train_acc = sum(tf_train_acc)/len(tf_train_acc)
tf_mean_val_acc = sum(tf_val_acc)/len(tf_val_acc)
tf_mean_test_acc = sum(tf_test_acc)/len(tf_test_acc)

In [117]:
tf_mean_train_acc, tf_mean_val_acc, tf_mean_test_acc

(0.9955294042825699, 0.9028333276510239, 0.8947499999999999)

## BERT

In [7]:
import datasets
from sklearn.metrics import accuracy_score, f1_score
from transformers import BertTokenizerFast, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)
train = { 'text' : x_train, 'label' : y_train }
test = { 'text' : x_test, 'label' : y_test }
train_data = datasets.Dataset.from_dict(train)
test_data = datasets.Dataset.from_dict(test)
data = datasets.DatasetDict( {"train" : train_data, "test" : test_data } )

In [10]:
model_name_bert = "bert-base-uncased"
tokenizer = BertTokenizerFast.from_pretrained(model_name_bert)

In [11]:
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True)

tokenized_data = data.map(preprocess_function, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [22]:
model = BertForSequenceClassification.from_pretrained(model_name_bert, num_labels = 5)

loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at C:\Users\PCZ/.cache\huggingface\transformers\3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.37395cee442ab11005bcd270f3c34464dc1704b715b5d7d52b1a461abe3b9e4e
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_em

In [24]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"Accuracy": acc, "F1": f1}

In [26]:
training_args = TrainingArguments(
    output_dir = './Bert_Results',
    learning_rate = 2e-5,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    load_best_model_at_end = True,
    metric_for_best_model = "F1",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    num_train_epochs = 2,
    weight_decay = 0.01,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [27]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset = tokenized_data["train"],
    eval_dataset = tokenized_data["test"],
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [28]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running training *****
  Num examples = 4000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1000


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.942700,0.484115,0.843000,0.841882
2,0.273300,0.402365,0.877000,0.876178


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./Bert_Results\checkpoint-500
Configuration saved in ./Bert_Results\checkpoint-500\config.json
Model weights saved in ./Bert_Results\checkpoint-500\pytorch_model.bin
tokenizer config file saved in ./Bert_Results\checkpoint-500\tokenizer_config.json
Special tokens file saved in ./Bert_Results\checkpoint-500\special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text.
***** Running Evaluation *****
  Num examples = 1000
  Batch size = 8
Saving model checkpoint to ./Bert_Results\checkpoint-1000
Configuration saved in ./Bert_Results\checkpoint-1000\config.json
Model weights saved in ./Bert_Results\checkpoint-1000\pytorch_model.bin

TrainOutput(global_step=1000, training_loss=0.6079967956542969, metrics={'train_runtime': 10270.5837, 'train_samples_per_second': 0.779, 'train_steps_per_second': 0.097, 'total_flos': 365409431567856.0, 'train_loss': 0.6079967956542969, 'epoch': 2.0})